![alt text](./pageheader_rose2_babies.jpg)

# Leak-compensated SIPPV-VG ventilator data

This notebook imports the **slow_measurements_sippv_vg_1.pickle** and **slow_measurements_sippv_vg_2.pickle** pickle archives generated by the **SIPPV_VG.ipynb** notebook. It identifies and exports leak-compensated SIPPV-VG recordings to two pickle archives: **slow_measurements_sippv_vg_lc_1.ipynb** and **slow_measurements_sippv_vg_lc_2.ipynb**.


#### Author: Dr Gusztav Belteki

Contact: gbelteki@aol.com

## Importing the required libraries and setting options

In [ ]:
import IPython
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import os
import sys
import re
import pickle

from scipy import stats
from pandas import Series, DataFrame
from datetime import datetime, timedelta

%matplotlib inline

matplotlib.style.use('classic')
matplotlib.rcParams['figure.facecolor'] = 'w'
plt.rcParams['axes.grid'] = True

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [ ]:
print("Python version: {}".format(sys.version))
print("IPython version: {}".format(IPython.__version__))
print("pandas version: {}".format(pd.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("NumPy version: {}".format(np.__version__))
print("SciPy version: {}".format(sp.__version__))

### Import custom functions from own module

In [ ]:
from gb_loader import *
from gb_stats import *
from gb_transform import *
from gb_visualizer import *

### List and set the working directory and the directories to write out data

In [ ]:
# Topic of the Notebook which will also be the name of the subfolder containing results
TOPIC = 'VG'

# Name of the external hard drive
DRIVE = 'GUSZTI'

# Directory containing clinical and blood gas data
CWD = '/Users/guszti/ventilation_data'


# Serialised data will be read from this folder of hard drive
DIR_READ_1 = '/Volumes/%s/data_dump/draeger/%s' % (DRIVE, TOPIC)

# Directory on external drive to read the original ventilation data from
DIR_READ_2 = '/Volumes/%s/Draeger/service_evaluation_old' % DRIVE  

# Directory to write results and selected images to 
if not os.path.isdir('%s/%s/%s' % (CWD, 'Analyses', TOPIC)):
    os.makedirs('%s/%s/%s' % (CWD, 'Analyses', TOPIC))
DIR_WRITE = '%s/%s/%s' % (CWD, 'Analyses', TOPIC)


# Images and raw data will be written on an external hard drive
if not os.path.isdir('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)):
    os.makedirs('/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC))
DATA_DUMP = '/Volumes/%s/data_dump/%s' % (DRIVE, TOPIC)

In [ ]:
os.chdir(CWD)
os.getcwd()

In [ ]:
DIR_READ_1

In [ ]:
DIR_READ_2

In [ ]:
DIR_WRITE

In [ ]:
DATA_DUMP

## Import ventilator data 
Files ending with **`slow_measurement.csv`** contain ventilator data obtained with 1Hz (1/sec) sampling rate.

Imports processed `slow_measurements` data from pickle archive. This has been exported by the notebook **'SIPPV_VG_for_paper.pynb'**

In [ ]:
with open('%s/%s.pickle' % (DIR_READ_1, 'slow_measurements_sippv_vg_1'), 'rb') as handle:
    slow_measurements_1 = pickle.load(handle)
with open('%s/%s.pickle' % (DIR_READ_1, 'slow_measurements_sippv_vg_2'), 'rb') as handle:
    slow_measurements_2 = pickle.load(handle)

## Identify which recordings were leak-compensated and which were not

**'leak_comp'** = **VTmand_kg** - **VTemand_kg**

*  If the leak-compensation if off VTmand = VTemand. The targeted parameter is VTemand.
*  If leak-compesation is on, VTmand > VTemand. The targeted parameter is VTmand



In [ ]:
leak_comp_on = []
leak_comp_off = []


for recording in recordings:
    if slow_measurements[recording]['leak_comp'].mean() > 0.001:
        leak_comp_on.append(recording) 
    else:
        leak_comp_off.append(recording) 

In [ ]:
print(leak_comp_on)

In [ ]:
print('Leak compensation was ON for %d recordings' % len(leak_comp_on))

In [ ]:
print(leak_comp_off)

In [ ]:
print('Leak compensation was OFF for %d recordings' % len(leak_comp_off))

### Export the leak-compensated recordings and the clinical details to pickls archive

In [ ]:
rec1 = leak_comp_on[:(len(leak_comp_on)//2)]
rec2 = leak_comp_on[(len(leak_comp_on)//2):]

In [ ]:
slow_measurements_1 = { key: value for key, value in slow_measurements.items() if key in rec1}
with open('%s/%s.pickle' % (DATA_DUMP, 'slow_measurements_sippv_vg_lc_1'), 'wb') as handle:
    pickle.dump(slow_measurements_1, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
slow_measurements_2 = { key: value for key, value in slow_measurements.items() if key in rec2}
with open('%s/%s.pickle' % (DATA_DUMP, 'slow_measurements_sippv_vg_lc_2'), 'wb') as handle:
    pickle.dump(slow_measurements_2, handle, protocol=pickle.HIGHEST_PROTOCOL)